In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gurobipy

In [2]:
demand = pd.read_csv("demand.csv")
generator = pd.read_csv("generator_data.csv")
windCF = pd.read_csv("windCF.csv")
generator = generator.set_axis(["Num","Loc","Capacity","MinGen","FuelType","MinUp","MinDown","RR","SC","NLC","a","b"], axis = 1)

generator = generator.drop(generator[generator["FuelType"] == " BIT "].index)
generator = generator.drop(generator[generator["FuelType"] == " SUB "].index)



In [3]:
def optimize2(Scen, Wind_Cap):
    chosen_demand_ex = demand[demand["Scenarios"] == Scen]
    chosen_windCF_ex = windCF[windCF["Scenarios"] == Scen]
    
    MC      = np.array(generator['a'])   # first-order marginal generator cost
    MC2     = np.array(generator['b'])   # second-order marginal cost
    NLC     = np.array(generator['NLC'])  # no load cost
    SUC     = np.array(generator['SC'])  # start up cost
    GenMin  = np.array(generator['MinGen'])   # minium generation
    GenMax  = np.array(generator['Capacity'])  # maximum generation
    MinUp   = np.array(generator['MinUp'])
    MinDown = np.array(generator['MinDown'])
    RR      = np.array(generator['RR'])  # ramp rate
    LOAD    = np.array(chosen_demand_ex['Total']) # demand
    CF      = np.array(chosen_windCF_ex['WindCF']) # capacity factor
    W       = Wind_Cap # installed wind turbine nameplate power
    
    g = cp.Variable((65,96), nonneg = True)   # generator dispatch
    wt = cp.Variable(96, nonneg = True)   # wind energy dispatch
    u = cp.Variable((65,96), boolean = True)   # commitment status
    v = cp.Variable((65,96), boolean = True)   # start-up status
    z = cp.Variable((65,96), boolean = True)   # close status
    obj = cp.Minimize(sum(MC @ g + MC2 @ (g**2) + NLC @ u + SUC @ v))
    
    # Initialize an empty constraint set
    con_set_1 = []  

    # power balance constraint, supply equals demand
    con_set_1.append( LOAD == sum(g) + wt ) # demand balance constraint
    con_set_1.append( wt <= CF * W ) # wind electricity is less than the product of capacity factor and nameplate power

    # use a for loop to define the unit constriant over each time period
    for t in range(96):  # go through each period
        for i in range(65):  # go through each  generator
            con_set_1.append(g[i][t] <= GenMax[i] * u[i][t])  # maximum generation limits
            con_set_1.append(g[i][t] >= GenMin[i] * u[i][t])  # minimum generation limits
            con_set_1.append(v[i][t] + z[i][t] <= 1)  # a generator is off this hour could not be turned on in this hour, in order to solve the problem that the generators with 0 start up costs would be turned on every hour without running

    for t in range(96):
        for i in range(65):
            if t == 0:
                con_set_1.append(v[i][t] == u[i][t])
            else:
                con_set_1.append(g[i][t] - g[i][t-1] <= RR[i] + GenMin[i] * v[i][t])
                con_set_1.append(g[i][t] - g[i][t-1] >= -RR[i])
                con_set_1.append(v[i][t] - z[i][t] == u[i][t] - u[i][t-1])

    for t in range(96):
        for i in range(65):
            con_set_1.append(sum([v[i][t-k] for k in range(max(t-MinUp[i],1))]) <= u[i][t])
            con_set_1.append(sum([z[i][t-k] for k in range(max(t-MinDown[i],1))]) <= 1 - u[i][t])

    prob1 = cp.Problem(obj, con_set_1)
    prob1.solve(solver = "GUROBI", MIPGap = 0.05)
    
    GT = pd.DataFrame(g.value)
    UT = pd.DataFrame(u.value)
    VT = pd.DataFrame(v.value)
    
    daily_operation_cost = prob1.value / 4

    return [GT, UT, VT, daily_operation_cost]

In [4]:
Scen = 64

wind_CAP = [0,1000,2000,4000,8000,12000,16000, 20000, 24000, 28000, 32000]
GT_result = []
UT_result = []
VT_result = []
daily_operation_cost_result = []
actual_wind_CAP = []

for i_cap in wind_CAP:
    [A, B, C, D] = optimize2(Scen, i_cap)
    GT_result.append(A)
    UT_result.append(B)
    VT_result.append(C)
    daily_operation_cost_result.append(D)
    actual_wind_CAP.append(i_cap)

Academic license - for non-commercial use only - expires 2022-10-27
Using license file /Users/shentianxiao/gurobi.lic


In [5]:
for i in range(len(GT_result)):    
    GT_result[i].to_csv(str("%.0f_GT2_%.0f.csv"% (Scen,actual_wind_CAP[i])))

In [6]:
for i in range(len(UT_result)):    
    UT_result[i].to_csv(str("%.0f_UT2_%.0f.csv"% (Scen,actual_wind_CAP[i])))
    VT_result[i].to_csv(str("%.0f_VT2_%.0f.csv"% (Scen,actual_wind_CAP[i])))

In [7]:
market_result = pd.DataFrame()
market_result["operating cost"] = daily_operation_cost_result

In [8]:
market_result.index = actual_wind_CAP

In [9]:
market_result.to_csv("%.0f_market_result2.csv"% Scen)